In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# additional imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/miran/.secret/yelp_api.json') as f:
    json_file = json.load(f)
type(json_file)

dict

In [3]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(json_file['api-key'], timeout_s=5.0)
yelp_api

In [4]:
json_file.keys()

dict_keys(['client-id', 'api-key'])

In [5]:
LOCATION = 'Gorham, NH'
TERM = 'food'

In [6]:
# Quick Test Query
results = yelp_api.search_query(location='Gorham, NH',
                                       term='food')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [7]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "/Users/miran/.secret/yelp_api.json"
JSON_FILE

'/Users/miran/.secret/yelp_api.json'

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] /Users/miran/.secret/yelp_api.json already exists.


In [9]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 2 previous results found.


In [10]:
total = results['total']
total

22

In [11]:
per_page = len(results['businesses'])
per_page

20

In [12]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ per_page)
n_pages

1

In [13]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2)

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [15]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [16]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] /Users/miran/.secret/yelp_api.json not found. Saving empty list to new file.
- 0 previous results found.


2

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,k3Q_83BFa_4x419nU7dalQ,big-day-brewing-gorham,Big Day Brewing,https://s3-media2.fl.yelpcdn.com/bphoto/JbMs_Q...,False,https://www.yelp.com/biz/big-day-brewing-gorha...,31,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 44.38637450548805, 'longitude': -...",[],"{'address1': '20 Glen Rd', 'address2': '', 'ad...",,,1342.441076,NaN
1,uJedD6K0GqJwjfmeJ9v5NQ,public-house-eatery-gorham,Public House Eatery,https://s3-media2.fl.yelpcdn.com/bphoto/3BB9wU...,False,https://www.yelp.com/biz/public-house-eatery-g...,36,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 44.392689, 'longitude': -71.187861}",[],"{'address1': '245 Main St', 'address2': '', 'a...",+16039159209,(603) 915-9209,53.516141,NaN
2,VKZrv5wQXmREQDSw6v49Qw,nonnas-kitchen-gorham,Nonna's Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/RBzcom...,False,https://www.yelp.com/biz/nonnas-kitchen-gorham...,86,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 44.38819, 'longitude': -71.17629}",[],"{'address1': '19 Exchange St', 'address2': Non...",+16039159203,(603) 915-9203,1001.196068,NaN
3,6elBFAGzM56ZGH7UDhamMw,js-corner-restaurant-and-lounge-gorham,J's Corner Restaurant & Lounge,https://s3-media4.fl.yelpcdn.com/bphoto/fTPYKC...,False,https://www.yelp.com/biz/js-corner-restaurant-...,115,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",4.0,"{'latitude': 44.3940221, 'longitude': -71.1908...",[],"{'address1': '277 Main St', 'address2': '', 'a...",+16034665132,(603) 466-5132,276.145276,$$
4,vhPLfjJbRjwply6e2JUyZQ,road-hawg-bbq-swine-dining-gorham,Road Hawg BBQ Swine Dining,https://s3-media4.fl.yelpcdn.com/bphoto/6GLjDZ...,False,https://www.yelp.com/biz/road-hawg-bbq-swine-d...,96,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...",4.0,"{'latitude': 44.391285, 'longitude': -71.185014}",[],"{'address1': '195 Main St', 'address2': '', 'a...",+16039159242,(603) 915-9242,247.307339,$$


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
17,OqPkIGlNOK7_Drv_LzgEdg,subway-gorham-3,Subway,https://s3-media3.fl.yelpcdn.com/bphoto/poUByq...,False,https://www.yelp.com/biz/subway-gorham-3?adjus...,7,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",1.5,"{'latitude': 44.3885439839016, 'longitude': -7...",[],"{'address1': '66 Main St', 'address2': 'Ste 1'...",+16034665200,(603) 466-5200,1160.535224,$
18,MRPyHJRB3TswDxqkBcwZFA,berlin-marketplace-berlin,Berlin Marketplace,,False,https://www.yelp.com/biz/berlin-marketplace-be...,3,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.0,"{'latitude': 44.46584, 'longitude': -71.18595}",[],"{'address1': '19 Pleasant St', 'address2': '',...",+16037521050,(603) 752-1050,8137.844478,$$
19,8dNZy2CZ_GSWw2WOveKPZg,cumberland-farms-gorham-3,Cumberland Farms,https://s3-media1.fl.yelpcdn.com/bphoto/WQ0afj...,False,https://www.yelp.com/biz/cumberland-farms-gorh...,5,"[{'alias': 'convenience', 'title': 'Convenienc...",4.0,"{'latitude': 44.388512, 'longitude': -71.173044}",[],"{'address1': '60 Main St', 'address2': '', 'ad...",+16034662157,(603) 466-2157,1228.597200,$
20,UIcvkRNMKmvEprf37RTxnQ,dunkin-gorham-2,Dunkin',https://s3-media2.fl.yelpcdn.com/bphoto/YRRhNf...,False,https://www.yelp.com/biz/dunkin-gorham-2?adjus...,13,"[{'alias': 'donuts', 'title': 'Donuts'}, {'ali...",1.5,"{'latitude': 44.39119677992125, 'longitude': -...",[],"{'address1': '191 Main St', 'address2': '', 'a...",+16034662979,(603) 466-2979,282.482354,$
21,pGWUkDvukwgWGGgZTs0FOA,circle-k-gorham,Circle K,https://s3-media4.fl.yelpcdn.com/bphoto/HigsHJ...,False,https://www.yelp.com/biz/circle-k-gorham?adjus...,1,"[{'alias': 'convenience', 'title': 'Convenienc...",1.0,"{'latitude': 44.3964885, 'longitude': -71.1940...",[],"{'address1': '350 Main St', 'address2': '', 'a...",+16034662178,(603) 466-2178,701.030839,NaN


In [19]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [20]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_nh_food.csv.gz', compression='gzip',index=False)